<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_projects/blob/main/predict_judge/Predict_Jugment_wandb_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 0: loss, 1:win

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 33.7 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.2 MB/s eta 0:00:00


In [ ]:
!pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=322a5018c99a7bb6fadc25dde3ac272feaedb35dfe21133bdaea90a6f0f29a31
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 56


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Data/judge"

/content/drive/MyDrive/Data/judge


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    AdamW,
    get_linear_schedule_with_warmup,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from datasets import load_metric

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
print('--train--')
print(train_df.shape)
display(train_df.head())
print('--test--')
print(test_df.shape)
display(test_df.head())

--train--
(2478, 5)


,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


--test--
(1240, 4)


,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [ ]:
drop_col = ['first_party', 'second_party']
train_df = train_df.drop(drop_col, axis=1)
test_df = test_df.drop(drop_col, axis=1)

In [ ]:
train_df.isnull().sum()

ID                    0
facts                 0
first_party_winner    0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X = train_df['facts'].values.tolist()
y = train_df['first_party_winner'].values.tolist()

train_texts, val_texts, train_labels, val_labels  = train_test_split(X, y, test_size=.2, stratify=y)

In [ ]:
from tokenizers import Tokenizer

In [ ]:
# MODEL_NAME =  'distilbert-base-uncased'
#MODEL_NAME =  'bert-base-uncased'
# MODEL_NAME = 'roberta-base'
MODEL_NAME = 'nlpaueb/legal-bert-base-uncased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# tokenizer.tokenize(train_texts[0])

In [ ]:
test_texts = test_df['facts'].values.tolist()

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random, 'bayes'
    'metric': {
        'goal': 'maximize',
        'name': 'val_acc'},
    'parameters': {
        'learning_rate': {'max': 1e-3, 'min': 6e-6},
        'weight_decay':  {'values': [0.01, 0.03, 0.1, 0.3, 0.5]},
        'SchedulerType':{'values': ["linear", "cosine", "constant"]}
        }
    }

In [ ]:
# sweep config random learning rate
BATCH_SIZE=8
num_epochs =10
SchedulerType ='linear'
random_seed = 42

config_defaults = {
    'epochs': num_epochs,
    'batch_size': BATCH_SIZE,
    'seed' : random_seed,
    'optimizer' :'AdamW',
    'SchedulerType': SchedulerType

}



In [ ]:
entity = "solowosophia13"
sweep_id = wandb.sweep(sweep_config, entity= entity, project="predict_judge_hypersearch")

Create sweep with ID: 8hcrqqn3
Sweep URL: https://wandb.ai/solowosophia13/predict_judge_hypersearch/sweeps/8hcrqqn3


In [ ]:
class JudgeDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [ ]:
train_dataset = JudgeDataset(train_encodings, train_labels)
val_dataset = JudgeDataset(val_encodings, val_labels)
test_dataset = JudgeDataset(test_encodings)

In [ ]:
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,  num_labels=2)
model.to(device)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# def compute_metrics(p):
#   pred, labels = p
#   pred = np.argmax(pred, axis=1)

#   accuracy = accuracy_score(y_true=labels, y_pred=pred)
#   return {'accuracy':accuracy}

def compute_metrics_fn(eval_preds):
  metrics = dict()

  accuracy_metric = load_metric('accuracy')

  logits = eval_preds.predictions
  labels = eval_preds.label_ids
  preds = np.argmax(logits, axis=-1)

  metrics.update(accuracy_metric.compute(predictions=preds, references=labels))

  return metrics


In [ ]:
def train():
  with wandb.init(config=config_defaults):
    config = wandb.config

    training_args = TrainingArguments(
        output_dir ='results',
        learning_rate= config.learning_rate,
        num_train_epochs=num_epochs,  # number of training epochs
        per_device_train_batch_size=BATCH_SIZE, # batch size of training
        per_device_eval_batch_size=BATCH_SIZE, # batch size of evaluation
        push_to_hub=False,
        gradient_accumulation_steps = 4,
        weight_decay=config.weight_decay,
        fp16=True,
        report_to="wandb",
        lr_scheduler_type = config.SchedulerType,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        logging_strategy='epoch',
        load_best_model_at_end=True,


    )

    trainer = Trainer(
        model=model,                        # model
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training data
        eval_dataset=val_dataset,            # evaluation dataset
        compute_metrics=compute_metrics_fn,
    )

    trainer.train()

In [ ]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: yvhc5tem with config:
wandb: 	SchedulerType: linear
wandb: 	learning_rate: 0.0007245590989287603
wandb: 	weight_decay: 0.1
wandb: Currently logged in as: solowosophia13. Use `wandb login --relogin` to force relogin


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.684800,0.671261,0.665323
2,0.663800,0.640827,0.665323
3,0.665500,0.736824,0.665323
4,0.657100,0.643260,0.665323
5,0.649600,0.638029,0.665323
6,0.645200,0.643196,0.665323
7,0.655700,0.637727,0.665323
8,0.647900,0.644171,0.665323
9,0.646000,0.637466,0.665323
10,0.642000,0.637699,0.665323


eval/accuracy,▁▁▁▁▁▁▁▁▁▁
eval/loss,▃▁█▁▁▁▁▁▁▁
eval/runtime,▃▂▂█▁▂▁▆▁▂
eval/samples_per_second,▆▇▇▁█▇█▃█▇
eval/steps_per_second,▆▇▇▁█▇█▃█▇
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▅▅▃▂▂▃▂▂▁
train/total_flos,▁
train/train_loss,▁


wandb: Agent Starting Run: gakjmrw6 with config:
wandb: 	SchedulerType: linear
wandb: 	learning_rate: 2.2814982144875653e-05
wandb: 	weight_decay: 0.1


...

eval/accuracy,██████▁███
eval/loss,▃█▁▁▃▂▂▁▃▁
eval/runtime,▅▁▁▄▆▇▄▁█▂
eval/samples_per_second,▄██▄▂▂▅█▁▇
eval/steps_per_second,▄██▄▂▂▅█▁▇
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,▁▁▁▁▁▁▁▁▁▁
train/loss,▁▂▅▆█▆███▇
train/total_flos,▁
train/train_loss,▁


wandb: Agent Starting Run: awowlyf4 with config:
wandb: 	SchedulerType: linear
wandb: 	learning_rate: 0.0005059825854616872
wandb: 	weight_decay: 0.5


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.505600,0.752394,0.665323
2,0.423400,0.790982,0.665323
3,0.507900,1.082314,0.665323
4,0.559800,0.749152,0.665323
5,0.623900,0.672924,0.665323
6,0.639100,0.645565,0.665323
7,0.648600,0.638584,0.665323
8,0.640200,0.644941,0.665323
9,0.648000,0.637472,0.665323
10,0.644500,0.638062,0.665323


eval/accuracy,▁▁▁▁▁▁▁▁▁▁
eval/loss,▃▃█▃▂▁▁▁▁▁
eval/runtime,▅▄▃▂▅▁▅▂▃█
eval/samples_per_second,▄▅▆▇▃█▃▇▆▁
eval/steps_per_second,▄▅▆▇▃█▃▇▆▁
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,▄▁▄▅▇█████
train/total_flos,▁
train/train_loss,▁


In [ ]:
wandb.finish()

In [ ]:
# Make prediction
# raw_pred, _, _ = trainer.predict(test_dataset)
# Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
# y_pred

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')

In [ ]:
# submit['first_party_winner'] = y_pred
# submit.to_csv('./submit.csv', index=False)
# print('Done')